In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_loss2"
#model = VGG16_quant()
#model_name = "resnet20_quant"
model = VGG16()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    #adjust_list = [150, 225]
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride

In [ ]:
# Using Loss 2 without gammar

In [2]:
def train_loss2(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss1 = criterion(output, target)
        loss2 = model.features[0].weight.abs().sum()
        loss = loss1 + loss2
        
        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

In [3]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train_loss2(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/391]	Time 0.432 (0.432)	Data 0.317 (0.317)	Loss 169.7443 (169.7443)	Prec 7.031% (7.031%)
Epoch: [0][100/391]	Time 0.044 (0.048)	Data 0.002 (0.005)	Loss 89.3808 (105.0390)	Prec 13.281% (10.334%)
Epoch: [0][200/391]	Time 0.043 (0.046)	Data 0.002 (0.003)	Loss 84.6383 (96.0098)	Prec 11.719% (10.798%)
Epoch: [0][300/391]	Time 0.044 (0.046)	Data 0.002 (0.003)	Loss 84.1769 (92.4021)	Prec 8.594% (11.350%)
Validation starts
Test: [0/79]	Time 0.232 (0.232)	Loss 2.8950 (2.8950)	Prec 8.594% (8.594%)
 * Prec 8.640% 
best acc: 8.640000
Epoch: [1][0/391]	Time 0.635 (0.635)	Data 0.603 (0.603)	Loss 84.8737 (84.8737)	Prec 15.625% (15.625%)
Epoch: [1][100/391]	Time 0.048 (0.050)	Data 0.001 (0.007)	Loss 85.5941 (86.7607)	Prec 15.625% (14.093%)
Epoch: [1][200/391]	Time 0.049 (0.047)	Data 0.002 (0.005)	Loss 81.3280 (85.4044)	Prec 13.281% (14.222%)
Epoch: [1][300/391]	Time 0.044 (0.046)	Data 0.002 (0.004)	Loss 83.7578 (84.5311)	Prec 14.844% (14.600%)
Validation starts
Test: [0/79]	Time 0.450 (0.

Epoch: [15][100/391]	Time 0.045 (0.049)	Data 0.001 (0.006)	Loss 75.3399 (74.9917)	Prec 61.719% (69.454%)
Epoch: [15][200/391]	Time 0.046 (0.047)	Data 0.001 (0.004)	Loss 73.8083 (75.0672)	Prec 67.969% (69.181%)
Epoch: [15][300/391]	Time 0.042 (0.046)	Data 0.001 (0.003)	Loss 76.5393 (75.0383)	Prec 64.062% (69.391%)
Validation starts
Test: [0/79]	Time 0.395 (0.395)	Loss 1.1816 (1.1816)	Prec 59.375% (59.375%)
 * Prec 59.050% 
best acc: 59.050000
Epoch: [16][0/391]	Time 0.642 (0.642)	Data 0.616 (0.616)	Loss 74.7358 (74.7358)	Prec 70.312% (70.312%)
Epoch: [16][100/391]	Time 0.044 (0.051)	Data 0.002 (0.008)	Loss 76.0766 (76.1210)	Prec 77.344% (70.498%)
Epoch: [16][200/391]	Time 0.045 (0.048)	Data 0.001 (0.005)	Loss 77.9683 (75.8809)	Prec 69.531% (70.884%)
Epoch: [16][300/391]	Time 0.045 (0.047)	Data 0.001 (0.004)	Loss 74.5351 (75.7889)	Prec 69.531% (70.961%)
Validation starts
Test: [0/79]	Time 0.300 (0.300)	Loss 1.6139 (1.6139)	Prec 42.969% (42.969%)
 * Prec 44.240% 
best acc: 59.050000
Epoch

Epoch: [30][100/391]	Time 0.045 (0.050)	Data 0.002 (0.007)	Loss 74.3852 (75.3459)	Prec 79.688% (81.815%)
Epoch: [30][200/391]	Time 0.044 (0.047)	Data 0.002 (0.004)	Loss 76.0395 (75.0211)	Prec 81.250% (81.709%)
Epoch: [30][300/391]	Time 0.044 (0.046)	Data 0.001 (0.003)	Loss 74.1479 (75.0387)	Prec 83.594% (81.741%)
Validation starts
Test: [0/79]	Time 0.306 (0.306)	Loss 0.7987 (0.7987)	Prec 72.656% (72.656%)
 * Prec 69.200% 
best acc: 72.680000
Epoch: [31][0/391]	Time 0.430 (0.430)	Data 0.405 (0.405)	Loss 74.0618 (74.0618)	Prec 81.250% (81.250%)
Epoch: [31][100/391]	Time 0.043 (0.048)	Data 0.002 (0.006)	Loss 77.7499 (76.1090)	Prec 72.656% (82.580%)
Epoch: [31][200/391]	Time 0.045 (0.047)	Data 0.001 (0.004)	Loss 75.9820 (75.7855)	Prec 73.438% (82.241%)
Epoch: [31][300/391]	Time 0.048 (0.046)	Data 0.001 (0.003)	Loss 76.4949 (75.7176)	Prec 78.906% (81.964%)
Validation starts
Test: [0/79]	Time 0.285 (0.285)	Loss 1.3131 (1.3131)	Prec 59.375% (59.375%)
 * Prec 51.990% 
best acc: 72.680000
Epoch

Epoch: [45][100/391]	Time 0.040 (0.051)	Data 0.002 (0.008)	Loss 75.8217 (76.1720)	Prec 85.156% (86.587%)
Epoch: [45][200/391]	Time 0.045 (0.048)	Data 0.002 (0.005)	Loss 74.4033 (75.8885)	Prec 83.594% (86.598%)
Epoch: [45][300/391]	Time 0.046 (0.047)	Data 0.001 (0.004)	Loss 73.2858 (75.6128)	Prec 84.375% (86.228%)
Validation starts
Test: [0/79]	Time 0.296 (0.296)	Loss 0.5944 (0.5944)	Prec 78.125% (78.125%)
 * Prec 74.850% 
best acc: 79.570000
Epoch: [46][0/391]	Time 0.364 (0.364)	Data 0.338 (0.338)	Loss 74.4383 (74.4383)	Prec 87.500% (87.500%)
Epoch: [46][100/391]	Time 0.045 (0.048)	Data 0.001 (0.005)	Loss 74.9897 (75.8355)	Prec 86.719% (87.144%)
Epoch: [46][200/391]	Time 0.045 (0.046)	Data 0.002 (0.003)	Loss 76.7307 (75.7452)	Prec 86.719% (86.653%)
Epoch: [46][300/391]	Time 0.045 (0.046)	Data 0.001 (0.003)	Loss 75.3590 (75.6036)	Prec 85.938% (86.254%)
Validation starts
Test: [0/79]	Time 0.303 (0.303)	Loss 0.6852 (0.6852)	Prec 76.562% (76.562%)
 * Prec 71.930% 
best acc: 79.570000
Epoch

Epoch: [60][100/391]	Time 0.043 (0.048)	Data 0.002 (0.005)	Loss 73.0800 (75.1979)	Prec 88.281% (89.194%)
Epoch: [60][200/391]	Time 0.042 (0.046)	Data 0.001 (0.003)	Loss 73.2612 (75.7195)	Prec 91.406% (89.066%)
Epoch: [60][300/391]	Time 0.048 (0.046)	Data 0.002 (0.003)	Loss 74.0877 (75.5454)	Prec 92.188% (88.920%)
Validation starts
Test: [0/79]	Time 0.456 (0.456)	Loss 0.6158 (0.6158)	Prec 84.375% (84.375%)
 * Prec 76.360% 
best acc: 79.570000
Epoch: [61][0/391]	Time 0.520 (0.520)	Data 0.499 (0.499)	Loss 75.0007 (75.0007)	Prec 95.312% (95.312%)
Epoch: [61][100/391]	Time 0.045 (0.049)	Data 0.001 (0.007)	Loss 75.2908 (75.6253)	Prec 89.844% (88.946%)
Epoch: [61][200/391]	Time 0.045 (0.047)	Data 0.001 (0.004)	Loss 74.1117 (75.5298)	Prec 84.375% (88.654%)
Epoch: [61][300/391]	Time 0.044 (0.046)	Data 0.001 (0.003)	Loss 76.8234 (75.5590)	Prec 86.719% (88.621%)
Validation starts
Test: [0/79]	Time 0.526 (0.526)	Loss 0.9369 (0.9369)	Prec 70.312% (70.312%)
 * Prec 69.370% 
best acc: 79.570000
Epoch

Epoch: [75][100/391]	Time 0.043 (0.048)	Data 0.002 (0.005)	Loss 75.9732 (76.2877)	Prec 90.625% (90.927%)
Epoch: [75][200/391]	Time 0.045 (0.047)	Data 0.001 (0.003)	Loss 77.8171 (76.3568)	Prec 89.844% (90.672%)
Epoch: [75][300/391]	Time 0.046 (0.046)	Data 0.002 (0.003)	Loss 73.6144 (75.9635)	Prec 91.406% (90.570%)
Validation starts
Test: [0/79]	Time 0.314 (0.314)	Loss 1.0501 (1.0501)	Prec 68.750% (68.750%)
 * Prec 69.400% 
best acc: 80.930000
Epoch: [76][0/391]	Time 0.493 (0.493)	Data 0.465 (0.465)	Loss 77.3578 (77.3578)	Prec 86.719% (86.719%)
Epoch: [76][100/391]	Time 0.045 (0.049)	Data 0.001 (0.006)	Loss 76.1771 (75.6475)	Prec 96.094% (91.035%)
Epoch: [76][200/391]	Time 0.045 (0.047)	Data 0.002 (0.004)	Loss 77.6777 (76.2327)	Prec 92.188% (90.563%)
Epoch: [76][300/391]	Time 0.045 (0.046)	Data 0.002 (0.003)	Loss 76.7885 (76.3805)	Prec 89.844% (90.454%)
Validation starts
Test: [0/79]	Time 0.181 (0.181)	Loss 0.8737 (0.8737)	Prec 70.312% (70.312%)
 * Prec 68.490% 
best acc: 80.930000
Epoch

Epoch: [90][200/391]	Time 0.045 (0.047)	Data 0.002 (0.004)	Loss 9.0915 (9.1227)	Prec 96.094% (96.871%)
Epoch: [90][300/391]	Time 0.045 (0.046)	Data 0.001 (0.003)	Loss 9.1558 (9.2971)	Prec 94.531% (96.810%)
Validation starts
Test: [0/79]	Time 0.331 (0.331)	Loss 0.8024 (0.8024)	Prec 78.906% (78.906%)
 * Prec 75.770% 
best acc: 85.310000
Epoch: [91][0/391]	Time 0.510 (0.510)	Data 0.485 (0.485)	Loss 9.2182 (9.2182)	Prec 91.406% (91.406%)
Epoch: [91][100/391]	Time 0.044 (0.049)	Data 0.001 (0.006)	Loss 9.2166 (9.2531)	Prec 99.219% (97.045%)
Epoch: [91][200/391]	Time 0.045 (0.047)	Data 0.001 (0.004)	Loss 8.9961 (9.2791)	Prec 99.219% (96.937%)
Epoch: [91][300/391]	Time 0.045 (0.046)	Data 0.001 (0.003)	Loss 9.0388 (9.2169)	Prec 94.531% (97.026%)
Validation starts
Test: [0/79]	Time 0.226 (0.226)	Loss 0.7332 (0.7332)	Prec 81.250% (81.250%)
 * Prec 78.080% 
best acc: 85.310000
Epoch: [92][0/391]	Time 0.536 (0.536)	Data 0.513 (0.513)	Loss 9.0112 (9.0112)	Prec 96.875% (96.875%)
Epoch: [92][100/391]	

In [6]:
#PATH = "result/VGG16_quant/model_best.pth.tar"
PATH = "result/VGG16_loss2/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8531/10000 (85%)



In [7]:
print(model.features[0].weight.abs().sum())

tensor(9.0050, device='cuda:0', grad_fn=<SumBackward0>)
